In [ ]:
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/objFuncs --break-system-packages
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/pyBO --break-system-packages

In [ ]:
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/objFuncs
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/pyBO

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
from phantasy import caget

In [ ]:
import objFuncs
from objFuncs import residuals
from objFuncs.util import plot_obj_history, plot_multi_obj_history

# check machineIO, source and beam 

In [ ]:
objFuncs._global_machineIO._test = False
objFuncs._ensure_set_timeout = 60
objFuncs._global_machineIO._fetch_data_time_span = 3.0
objFuncs._global_machineIO._ensure_set_timewait_after_ramp : 0.25
objFuncs._global_machineIO.view()

In [ ]:
SCS = caget("ACS_DIAG:DEST:ACTIVE_ION_SOURCE")
ion = caget("FE_ISRC"+str(SCS)+":BEAM:ELMT_BOOK")
# Q = caget("FE_ISRC"+str(SCS)+":BEAM:Q_BOOK")
Q = caget("ACC_OPS:BEAM:Q_STRIP")
A = caget("FE_ISRC"+str(SCS)+":BEAM:A_BOOK")
# AQ = caget("FE_ISRC2:BEAM:MOVRQ_BOOK")
AQ = A/Q
ion = str(A)+ion+str(Q)
print(ion, 'A/Q=',AQ)

# preprare objective (for residual construction)

In [ ]:
now0 = datetime.datetime.now()
fname = now0.strftime('%Y%m%d_%H%M')+'['+ion+'][LSQ][FS1][before1stDipole]QuadCentering'
fname

In [ ]:
decision_CSETs=[
#     'FS1_CSS:PSC2_D2276:I_CSET',
#     'FS1_CSS:PSC1_D2276:I_CSET',
    'FS1_CSS:PSC2_D2351:I_CSET',
    'FS1_CSS:PSC1_D2351:I_CSET',
    'FS1_CSS:PSC2_D2367:I_CSET',
    'FS1_CSS:PSC1_D2367:I_CSET'
]
decision_min = [-2*AQ]*len(decision_CSETs)
decision_max = [ 2*AQ]*len(decision_CSETs)

print(f'decision_min: {decision_min}')
print(f'decision_max: {decision_max}')

In [ ]:
objective_goal = { 
    'FS1_BBS:BPM_D2421:XPOS_RD' : 0.0,
    'FS1_BBS:BPM_D2421:YPOS_RD' : 0.0,
    'FS1_BBS:BPM_D2466:XPOS_RD' : 0.0,
    'FS1_BBS:BPM_D2466:YPOS_RD' : 0.0,
    'FS1_BMS:BPM_D2502:XPOS_RD' : 0.0,
    'FS1_BMS:BPM_D2502:YPOS_RD' : 0.0,
}
objective_norm = {pv:1.0 for pv in objective_goal.keys()}
objective_weight = {pv:1.0 for pv in objective_goal.keys()}
objective_RDs = list(objective_goal.keys())

In [ ]:
conditional_SETs = {
#     'FS1_CSS:PSQ_D2356:I_CSET':[],
#     'FS1_CSS:PSQ_D2362:I_CSET':[],
    'FS1_CSS:PSQ_D2372:I_CSET':[],
    'FS1_CSS:PSQ_D2377:I_CSET':[],
}

In [ ]:
# 3-point Q-scan
# for i,PV in enumerate(conditional_SETs.keys()):
#     x0 = caget(PV)
#     conditional_SETs[PV] = [x0]*(2*len(conditional_SETs)+1)
#     conditional_SETs[PV][2*i+1] = x0 - 5*AQ
#     conditional_SETs[PV][2*i+2] = x0 + 5*AQ

In [ ]:
# 2-point Q-scan
for i,PV in enumerate(conditional_SETs.keys()):
    x0 = caget(PV)
    conditional_SETs[PV] = [x0]*(len(conditional_SETs)+1)
    conditional_SETs[PV][i+1] = x0 + 10*AQ

In [ ]:
n_condition = len(list(conditional_SETs.values())[0])
conditional_SETs

In [ ]:
res = residuals.residualObjMultiConditionalVar(  
    decision_CSETs = decision_CSETs,
    decision_min = decision_min,
    decision_max =  decision_max,
    objective_goal = objective_goal,
    objective_norm = objective_norm,
    objective_weight = objective_weight,
    conditional_SETs = conditional_SETs,
    var_obj_weight_fraction = 1.0,
    )

In [ ]:
plot_decision = plot_obj_history(
    res.history['decision_CSETs'],
    title='decision_CSETs',
)
plot_conditions_RDs = plot_multi_obj_history(
    histories = [res.history['condition'+str(i)]['objective_RDs'] for i in range(n_condition)],
    history_labels = ['C'+str(i) for i in range(n_condition)],
    title='multi-condition_obj_RDs',
)
plot_objectives_var = plot_obj_history(
    res.history['objectives_var'],
    title='objectives_var',
)
plot_cond_control = plot_obj_history(
    res.history['condition controller']['decision_CSETs'],
    title='condition controller',
)
callbacks = [plot_decision,plot_conditions_RDs,plot_objectives_var,plot_cond_control]
# callbacks = []

In [ ]:
result = res.lsq_linear(jac_use3points=True,callbacks=callbacks)
res.condition_controller._set_decision(res.condition_controller.x0)
for f in callbacks:
    f.close()

In [ ]:
objectives_var_hist = [np.sum(res.history['objectives_var']['values'][i]**2) for i in range(len(res.history['objectives_var']['values']))]
imin = np.argmin(objectives_var_hist)
x_best = res.history['decision_CSETs']['values'][imin]
res(x_best)

In [ ]:
res.condition_controller._set_decision(res.condition_controller.x0)
res.write_log(fname=os.path.join('./log',fname))

In [ ]:
objectives_var_hist